In [79]:
import os
import sys
sys.path.append(r'C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\ADV')
from vector import Vector
from datetime import datetime


In [80]:
# location of raw data
dataFolder = r'O:\HybridDune experiment\data ADV, OBS\ADV RWS1\Deployment 1, until 23dec'
dataFolder = r'O:\HybridDune experiment\data ADV, OBS\ADV RWS2\Deployment 1, until 23dec'
dataFolder = r'O:\HybridDune experiment\data ADV, OBS\ADV RWS3\S3, Deployment1, until dec23'
dataFolder = r'O:\HybridDune experiment\data ADV, OBS\ADV RWS4\Deployment until 23dec'
dataFolder = r'O:\HybridDune experiment\data ADV, OBS\ADV TUD10\Deployment until 23dec'

# name of the instantiated vector class
name = 'ADV_RWS4_Deployment1'
name = 'ADV_TUD1_Deployment1'

# start time over which to read data (must be larger than first recorded time)
tstart = '2024-12-16 13:00:00'
tstart = '2024-12-13 10:00:00'

# stop time over which to read data (must be smaller than last recorded time)
tstop = '2024-12-23 13:00:00'
tstop = '2024-12-23 19:00:00'

# make map 'raw_netcdf' in dataFolder and set this as output directory
filename_out = 'ADV_RWS4_Deployment1.nc'
filename_out = 'ADV_TUD1_Deployment1.nc'

ncOutDir = r'O:\HybridDune experiment\data ADV, OBS\raw NetCDF'

if not os.path.exists(ncOutDir):
    os.makedirs(ncOutDir)


In [ ]:
# raw data to netcdf
vec = Vector(name, dataFolder, tstart=tstart, tstop=tstop)

# reads the raw data from tstart to tstop and casts all data in a pandas DataFrame that is stored under vec.dfpuv.
# in case there is no data between tstart and tstop the DataFrame is not instantiated
vec.read_raw_data()

# break up the data into burst blocks
vec.cast_to_blocks_in_xarray(blockWidth=1200) # 1200 seconds = 20 minutes  # Daan: blocks turn out 1 hour. Maybe because every hour misses 1 minute/10s/something? CHECK!

# compute burst averages (make sure to read vector.py what is happening exactly!)
vec.compute_block_averages()

# all data is collected in an xarray Dataset ds. We extract this from the class instantiation and
# we can easily write it to netCDF
ds = vec.ds

# add global attribute metadata
ds.attrs = {'Conventions': 'CF-1.6',
            'title': '{}'.format(vec.name),
            'instrument': '{}'.format('vec1'),
            'instrument serial number': '{}'.format(16725),
            'epsg': 28992,
             'x': 117196.6,
            'y': 559818.2,
            'time zone': 'UTC+2',
            'coordinate type': 'XYZ',
            'summary': 'hybrid-Dune field campaign',
            'contact person': 'Ruben Rosman ',
            'emailadres': 'r.g.c.rosman@student.tudelft.nl',
            'construction datetime': datetime.now().strftime("%d-%b-%Y (%H:%M:%S)"),
            'version': 'v1',
            'version comments': 'constructed with xarray'}

# Save the dataset to netCDF --------------------------------------------------------
# First define a custom encoding dictionary for the ADV variables, to save variables with the same accuracy (same number of decimals) as the original text file with data
encoding = {'p': { 'scale_factor': 10, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'u': { 'scale_factor': 0.001, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},  # three decimals originally, so scale factor 0.001. max value is 7m/s, with 3 decimals is 7000 options, so int16 scale of ± 32767 is sufficient
            'v': { 'scale_factor': 0.001, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},  # shuffle: flag for bit order. I just tried for which variables it saves data. (default is True when using deflate compression)
            'w': { 'scale_factor': 0.001, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'anl1': { 'dtype': 'uint16'},
            'anl2': { 'dtype': 'uint16'},
            'a1': { 'dtype': 'int16', '_FillValue': -999},
            'a2': { 'dtype': 'int16', '_FillValue': -999},
            'a3': { 'dtype': 'int16', '_FillValue': -999},
            'cor1': { 'dtype': 'int8', '_FillValue': -99},
            'cor2': { 'dtype': 'int8', '_FillValue': -99},
            'cor3': { 'dtype': 'int8', '_FillValue': -99},
            'snr1': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'snr2': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'snr3': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'voltage': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'heading': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999},
            'pitch': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'roll': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999, 'shuffle': False},
            'burst': { 'scale_factor': 0.1, 'dtype': 'int16', '_FillValue': -999} }

# Then add deflate compression level 4 to all variables and coordinates in netCDF, without overwriting existing keys
compression = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # temporary dict, with only compression settings
for var, comp in compression.items():  # for each variable in the dataset, 
    if var in encoding:                # if the variable already has an encoding, update it with the compression settings
        encoding[var].update(comp)
    else:                              # if the variable does not have an encoding yet, add it 
        encoding[var] = comp
encoding

ds.to_netcdf(os.path.join(ncOutDir, filename_out), encoding=encoding)
#

C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\ADV\vector.py:219: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  self.burstStartTimes = pd.date_range(start = self.tstart,periods = nBursts,


.dat file was read


C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\ADV\vector.py:288: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  timeSen = pd.date_range(start = self.tstart, periods = nSamples,


.sen file was read


C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\ADV\vector.py:323: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df2 = df2.resample('{}S'.format(1/self.frequency)).asfreq()
C:\Users\dpoppema\Documents\GitHub\HybridDune\Ruben\ADV\vector.py:330: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df3 = df3.fillna(method='ffill')
C:\Users\dpoppema\AppData\Local\Temp\ipykernel_16736\2669706652.py:67: SerializationWarning: saving variable anl1 with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf(os.path.join(ncOutDir, filename_out), encoding=encoding)
C:\Users\dpoppema\AppData\Local\Temp\ipykernel_16736\2669706652.py:67: SerializationWarning: saving variable anl2 with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf(os.path.join(ncOutDir, filename_out), encodi

In [82]:
# ds.p.plot()
